# Tutorial-Fast mode data processing

This tutorial explains the steps to be followed to process the data that has been generated for the
fast calculation mode. In this example, the fast mode data for"Air_30kW" model is processed.

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
with open("../../mosaik_components/heatpump/cop_m_data.json") as feedsjson:
    feeds = json.load(feedsjson)

In [ ]:
df = pd.DataFrame.from_dict(feeds['Air_30kW'], orient='index')
# print(df)

In [ ]:
df1 = df.copy()

In [ ]:
def dict_to_df(sub_dict1):
    sub_df1 = pd.DataFrame.from_dict(sub_dict1, orient='index')
    sub_df1.replace(0,np.nan,inplace=True)
    return sub_df1

In [ ]:
df1 = df1.applymap(dict_to_df)


In [ ]:
def data_fill_func(sub_df1):
    sub_df1.interpolate(limit=2,limit_area='inside',inplace=True)
    kw = dict(fill_value="extrapolate", limit_direction="both")
    sub_df1.interpolate(inplace=True, **kw)
    return sub_df1

In [ ]:
df1=df1.applymap(data_fill_func)
df1.index = df1.index.astype(float)
df1.columns = df1.columns.astype(float)

In [ ]:
df2 = df1.copy()

In [ ]:
def check_if_empty(df_chk):
    return df_chk.isna().all()[0]

def df_data_fill(sub_df, idx, main_df):
    # print(idx)
    col=sub_df['index']
    col_min = min(main_df.columns)
    col_max = max(main_df.columns)
    idx_min = min(main_df.index)
    idx_max = max(main_df.index)
    if not sub_df[1].empty:
        if check_if_empty(sub_df[1]):
            skip = False
            col_new = col -1
            if col_new < col_min:
                col_new = col_min
            sub_df_new = main_df.loc[idx, col_new]
            if not sub_df_new.empty:
                if not check_if_empty(sub_df_new) and sub_df[1].index.equals(sub_df_new.index):
                    sub_df[1] = sub_df_new
                    skip = True
            if not skip:
                col_new = col + 1
                if col_new > col_max:
                    col_new = col_max
                sub_df_new = main_df.loc[idx, col_new]
                if not sub_df_new.empty:
                    if not check_if_empty(sub_df_new) and sub_df[1].index.equals(sub_df_new.index):
                        sub_df[1] = sub_df_new
                        skip = True
            if not skip:
                idx_new = idx - 1
                if idx_new < idx_min:
                    idx_new = idx_min
                sub_df_new = main_df.loc[idx_new, col]
                if not sub_df_new.empty:
                    if not check_if_empty(sub_df_new) and sub_df[1].index.equals(sub_df_new.index):
                        sub_df[1] = sub_df_new
                        skip = True
            if not skip:
                idx_new = idx + 1
                if idx_new > idx_max:
                    idx_new = idx_max
                sub_df_new = main_df.loc[idx_new, col]
                if not sub_df_new.empty:
                    if not check_if_empty(sub_df_new) and sub_df[1].index.equals(sub_df_new.index):
                        sub_df[1] = sub_df_new

            main_df.at[idx, col] = sub_df[1]

    return sub_df[1]

In [ ]:
df2 = df2.apply(lambda x: x.reset_index().apply(lambda d: df_data_fill(d,idx=x.name, main_df=df1), axis=1), axis=1)

In [ ]:
df1.index = df1.index.astype(int)
df1.columns = df1.columns.astype(int)

In [ ]:
def df_fillna(sub_df1):
    sub_df1.fillna(0,inplace=True)
    return sub_df1

In [ ]:
df1 = df1.applymap(df_fillna)

In [ ]:
model_name = 'Air_30kW'
result = df1.to_json(orient="index")
parsed = json.loads(result)
json.dumps(parsed, indent=4)

In [ ]:
with open("../../mosaik_components/heatpump/cop_m_data.json") as feedsjson:
    feeds = json.load(feedsjson)

if model_name in feeds.keys():
    a = input('The model name already exists and must be replaced. Do you want to replace the data (y/n)? : ')
    if a.lower() == 'y':
        feeds[model_name]= parsed
    elif a.lower() =='n':
        model_name = input('Enter new model name')
        feeds[model_name]= parsed
    else:
        print("Wrong input. Execute cell again.")
else:
    feeds[model_name] = parsed

In [ ]:
with open("../../mosaik_components/heatpump/cop_m_data.json", "w") as write_file:
    json.dump(feeds, write_file, indent=4)